In [ ]:
import json

url = "https://hs-consumer-api.espncricinfo.com/v1/pages/match/comments?"

def GetOverData(overNumber, matchId, InningsNumber, seriesId, commentType = "ALL"):
    
    import requests
    
    request_url = url + f'seriesId={seriesId}&matchId={matchId}&inningNumber={InningsNumber}&commentType={commentType}&fromInningOver={overNumber}'
    
    payload={}
    headers = {}
    #print("url: ", request_url)
    response = requests.request("GET", request_url, headers=headers, data=payload)

    #print(request_url);
    
    return response.text

In [ ]:
def CheckValidResponse(body):
    j = json.loads(body)
    #print("No error:" ,not ("error" in j))
    #print(j)
    return not ("error" in j)
        
def CheckEmptyRequest(body):
    j = json.loads(body)
    #print ("nextInningsNotNull",j["nextInningOver"])
    return j["nextInningOver"] == None

def CheckResponse(body):
    return CheckValidResponse(body) and not CheckEmptyRequest(body)


def ExtractComments(body):
    j = json.loads(body)
    comments = []
    if(not "comments" in j):
        return comments
    
    for i,j2 in enumerate(j["comments"]):
        if(not "commentTextItems" in j2 or j2["commentTextItems"] == None):
            print("skip")
            continue
        #print(j2)
        comment = j2["commentTextItems"][0]["html"]
        comments.append(comment)
    return comments

In [ ]:
def GetAllDataInnings(matchId, InningsNumber, seriesId, commentType = "ALL"):
    InningsComments = ["start"]
    over = 3
    
    while(CheckResponse(responseText := GetOverData(over, matchId, InningsNumber, seriesId, commentType))):
        print("over: ", over)
        comments = ExtractComments(responseText)
        InningsComments.extend(comments)
        over += 2
    return InningsComments
       
def GetMatchData(MatchId, seriesId, commentType = "ALL"):
    Comments = []
    for innings in [1,2,3,4]:
        print("\t\t Innings", innings)
        Comments.extend(GetAllDataInnings(MatchId, innings, seriesId, commentType))
                        
    return Comments                    
    
def ExtractMatchIdsSeriesIds(lines):
    Ids = []
    for line in lines:
        temp = line.split(" ")
        t = (temp[0],temp[1]) #tuple
        Ids.append(t)
    return Ids
                        
def GetAllCommentsFromDoc(filename):
    file = open(filename)
    lines = file.read().splitlines()
    file.close()
    Ids = ExtractMatchIdsSeriesIds(lines)
    Comments = []
    for matchId, SeriesId in Ids:
        Comments.extend(GetMatchData(matchId, SeriesId))
                        
    return Comments

def WriteCommentsToFile(comments):
    file = open("AllComments.txt","w")
    file.write("\n".join(comments))
    file.close()

In [ ]:
allComments = GetAllCommentsFromDoc("ashesMatchIds.txt")
WriteCommentsToFile(allComments)